In [ ]:
!pip install catboost

In [2]:
# 2: Import Libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import gc

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [3]:
# 3: Local Validation Metric (Winkler Score)
# This remains the same. A reliable local score is critical.
def winkler_score(y_true, lower, upper, alpha=0.1):
    score = upper - lower
    if y_true < lower:
        score += (2 / alpha) * (lower - y_true)
    elif y_true > upper:
        score += (2 / alpha) * (y_true - upper)
    return score

def mean_winkler_score(y_true, lower, upper, alpha=0.1):
    scores = [winkler_score(yt, l, u, alpha) for yt, l, u in zip(y_true, lower, upper)]
    return np.mean(scores) 

In [ ]:
# 4: Data Loading and Memory Optimization
# This also remains the same.
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

try:
    from pathlib import Path
    
    # Define project_root as the current working directory
    project_root = Path.cwd()
    base_dir = project_root / "data"

    train_path = base_dir / "dataset.csv"
    test_path  = base_dir / "test.csv"

    train_df_raw = pd.read_csv(train_path)
    test_df      = pd.read_csv(test_path)

    train_df_raw = reduce_mem_usage(train_df_raw)
    test_df = reduce_mem_usage(test_df)

    print("Data loaded and memory reduced.")

except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    train_df_raw = pd.DataFrame()
    test_df = pd.DataFrame()

In [ ]:
# 5: Exploratory Data Analysis (EDA) and Visualizations
if not train_df_raw.empty:
    # 5.1. Target Variable Analysis
    plt.figure(figsize=(18, 6))

    plt.subplot(1, 2, 1)
    sns.histplot(train_df_raw['sale_price'], kde=True, bins=50)
    plt.title('Distribution of Original Sale Price')
    plt.xlabel('Sale Price')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    sns.histplot(np.log1p(train_df_raw['sale_price']), kde=True, bins=50)
    plt.title('Distribution of Log-transformed Sale Price')
    plt.xlabel('Log(Sale Price + 1)')
    plt.ylabel('Frequency')

    plt.suptitle('Target Variable Distribution Analysis', fontsize=16)
    plt.show()



In [ ]:
    # 5.2. Correlation Analysis
    df_train_for_corr = train_df_raw.copy()
    df_train_for_corr['log_sale_price'] = np.log1p(df_train_for_corr['sale_price'])

    corr_matrix = df_train_for_corr.select_dtypes(include=np.number).corr()

    top_corr_features = corr_matrix['log_sale_price'].abs().sort_values(ascending=False).index[1:21]

    plt.figure(figsize=(12, 10))
    sns.barplot(x=corr_matrix['log_sale_price'][top_corr_features], y=top_corr_features)
    plt.title('Top 20 Features Correlated with Log-transformed Sale Price')
    plt.xlabel('Correlation Coefficient')
    plt.ylabel('Feature Name')
    plt.show()

In [ ]:
# 6: Feature Engineering and Preprocessing
def feature_engineer(df):
    df_copy = df.copy()
    # Use a flexible format and coerce errors to NaT (Not a Time)
    df_copy['sale_date'] = pd.to_datetime(df_copy['sale_date'], format='mixed', errors='coerce')

    # Handle any dates that could not be parsed
    # A simple and robust strategy is to fill with the median date from the parsed dates
    if df_copy['sale_date'].isnull().any():
        median_date = df_copy['sale_date'].median()
        df_copy['sale_date'].fillna(median_date, inplace=True)

    df_copy['sale_year'] = df_copy['sale_date'].dt.year
    df_copy['sale_month'] = df_copy['sale_date'].dt.month
    df_copy['sale_dayofyear'] = df_copy['sale_date'].dt.dayofyear
    df_copy['house_age'] = df_copy['sale_year'] - df_copy['year_built']
    df_copy['years_since_reno'] = np.where((df_copy['year_reno'] > 0) & (df_copy['year_reno'] >= df_copy['year_built']), df_copy['sale_year'] - df_copy['year_reno'], 0)
    df_copy['was_renovated'] = (df_copy['years_since_reno'] > 0).astype(int)
    df_copy = df_copy.drop(columns=['sale_date', 'year_built', 'year_reno'])
    return df_copy

if not train_df_raw.empty:
    train_df_raw['log_sale_price'] = np.log1p(train_df_raw['sale_price'])
    TARGET = 'log_sale_price'
    FEATURES = [col for col in train_df_raw.columns if col not in ['id', 'sale_price', 'log_sale_price']]

    X = train_df_raw[FEATURES]
    y = train_df_raw[TARGET]
    X_test = test_df[[col for col in FEATURES if col in test_df.columns]]

    X = feature_engineer(X)
    X_test = feature_engineer(X_test)

    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

    # CRITICAL FIX: Fill NaN values in categorical features before type conversion.
    # CatBoost cannot handle NaN in categorical columns.
    for col in categorical_features:
        X[col] = X[col].fillna('missing')
        X_test[col] = X_test[col].fillna('missing')

    for col in categorical_features:
        X[col] = X[col].astype('category')
        X_test[col] = X_test[col].astype('category')

    preprocessor = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
    X[numerical_features] = preprocessor.fit_transform(X[numerical_features])
    X_test[numerical_features] = preprocessor.transform(X_test[numerical_features])

    print("Preprocessing complete.")
    del train_df_raw
    gc.collect()

In [ ]:
# 7: Data Splitting for Conformal Prediction
if 'X' in locals() and not X.empty:
    X_train_proper, X_calib, y_train_proper, y_calib = train_test_split(X, y, test_size=0.2, random_state=42)
    print("Data split into training and calibration sets.")

In [ ]:
# 8: Model Training
# Train all 6 models (3 base models x 2 quantiles).
if 'X_train_proper' in locals():
    alpha = 0.1
    q_lower = alpha / 2.0
    q_upper = 1 - (alpha / 2.0)

    # --- LightGBM ---
    print("Training LightGBM models...")
    lgbm_params = {'objective': 'quantile', 'metric': 'quantile', 'random_state': 42, 'n_estimators': 1000}
    lgbm_lower = lgb.LGBMRegressor(alpha=q_lower, **lgbm_params)
    lgbm_upper = lgb.LGBMRegressor(alpha=q_upper, **lgbm_params)
    lgbm_lower.fit(X_train_proper, y_train_proper, categorical_feature=categorical_features, eval_set=[(X_calib, y_calib)], callbacks=[lgb.early_stopping(100, verbose=False)])
    lgbm_upper.fit(X_train_proper, y_train_proper, categorical_feature=categorical_features, eval_set=[(X_calib, y_calib)], callbacks=[lgb.early_stopping(100, verbose=False)])

    # --- XGBoost ---
    print("Training XGBoost models...")
    # Convert categorical features to integer codes for XGBoost
    X_train_xgb = X_train_proper.copy()
    X_calib_xgb = X_calib.copy()
    for col in categorical_features:
        X_train_xgb[col] = X_train_xgb[col].cat.codes
        X_calib_xgb[col] = X_calib_xgb[col].cat.codes
    xgb_params = {'objective': 'reg:quantileerror', 'random_state': 42, 'n_estimators': 1000, 'tree_method': 'hist'}
    xgb_lower = xgb.XGBRegressor(quantile_alpha=q_lower, **xgb_params)
    xgb_upper = xgb.XGBRegressor(quantile_alpha=q_upper, **xgb_params)
    # This simplified fit call is more robust across different environments.
    xgb_lower.fit(X_train_xgb, y_train_proper, eval_set=[(X_calib_xgb, y_calib)], verbose=False)
    xgb_upper.fit(X_train_xgb, y_train_proper, eval_set=[(X_calib_xgb, y_calib)], verbose=False)

    # --- CatBoost ---
    print("Training CatBoost models...")
    # Option 1: Disable logging completely by setting train_dir=None
    cat_params_lower = {'loss_function': f'Quantile:alpha={q_lower}', 'iterations': 1000, 'verbose': 0, 'random_seed': 42, 'train_dir': None}
    cat_params_upper = {'loss_function': f'Quantile:alpha={q_upper}', 'iterations': 1000, 'verbose': 0, 'random_seed': 42, 'train_dir': None}

    # Option 2: Or specify a custom directory (uncomment the lines below and comment out the ones above)
    # cat_params_lower = {'loss_function': f'Quantile:alpha={q_lower}', 'iterations': 1000, 'verbose': 0, 'random_seed': 42, 'train_dir': './ml_model/catboost_logs'}
    # cat_params_upper = {'loss_function': f'Quantile:alpha={q_upper}', 'iterations': 1000, 'verbose': 0, 'random_seed': 42, 'train_dir': './ml_model/catboost_logs'}

    cat_lower = cb.CatBoostRegressor(**cat_params_lower)
    cat_upper = cb.CatBoostRegressor(**cat_params_upper)

    cat_lower.fit(X_train_proper, y_train_proper, cat_features=categorical_features, eval_set=[(X_calib, y_calib)], early_stopping_rounds=100)
    cat_upper.fit(X_train_proper, y_train_proper, cat_features=categorical_features, eval_set=[(X_calib, y_calib)], early_stopping_rounds=100)

    print("All models trained.")

In [ ]:
# 9: Ensemble CQR Calibration and Evaluation
if 'lgbm_lower' in locals():
    # 1. Get predictions from all models on the calibration set
    lgbm_cal_preds_lower = lgbm_lower.predict(X_calib)
    lgbm_cal_preds_upper = lgbm_upper.predict(X_calib)
    xgb_cal_preds_lower = xgb_lower.predict(X_calib_xgb) # Use encoded data for XGBoost
    xgb_cal_preds_upper = xgb_upper.predict(X_calib_xgb) # Use encoded data for XGBoost
    cat_cal_preds_lower = cat_lower.predict(X_calib)
    cat_cal_preds_upper = cat_upper.predict(X_calib)

    # 2. Average the predictions to create the ensemble interval
    ensemble_cal_lower = np.mean([lgbm_cal_preds_lower, xgb_cal_preds_lower, cat_cal_preds_lower], axis=0)
    ensemble_cal_upper = np.mean([lgbm_cal_preds_upper, xgb_cal_preds_upper, cat_cal_preds_upper], axis=0)

    # 3. Calculate conformity scores for the ENSEMBLE
    conformity_scores = np.maximum(ensemble_cal_lower - y_calib, y_calib - ensemble_cal_upper)

    # 4. Calculate the correction term 'q'
    n_calib = len(X_calib)
    q_level = np.ceil((1 - alpha) * (n_calib + 1)) / n_calib
    q = np.quantile(conformity_scores, q_level, interpolation='higher')
    print(f"Calculated Ensemble CQR correction factor (q): {q:.4f}")

    # 5. Apply correction and evaluate
    final_lower_bound_log = ensemble_cal_lower - q
    final_upper_bound_log = ensemble_cal_upper + q

    y_calib_orig = np.expm1(y_calib)
    final_lower_bound_orig = np.expm1(final_lower_bound_log)
    final_upper_bound_orig = np.expm1(final_upper_bound_log)

    coverage = np.mean((y_calib_orig >= final_lower_bound_orig) & (y_calib_orig <= final_upper_bound_orig))
    winkler = mean_winkler_score(y_calib_orig, final_lower_bound_orig, final_upper_bound_orig)

    print(f"\nTarget Coverage: {1-alpha:.2f}")
    print(f"Ensemble Empirical Coverage on Hold-out Set: {coverage:.4f}")
    print(f"Ensemble Mean Winkler Score on Hold-out Set: {winkler:,.2f}")

In [ ]:
# 10: Model Performance Visualization
if 'y_calib_orig' in locals():
    # 10.1. Prediction Interval Visualization
    results_df = pd.DataFrame({
        'y_true': y_calib_orig,
        'lower': final_lower_bound_orig,
        'upper': final_upper_bound_orig
    }).sort_values('y_true').reset_index(drop=True)

    sample_results = results_df.sample(n=min(500, len(results_df)), random_state=42).sort_values('y_true')

    plt.figure(figsize=(15, 8))
    plt.scatter(range(len(sample_results)), sample_results['y_true'], color='blue', label='True Values', s=15, zorder=3)
    plt.fill_between(
        range(len(sample_results)),
        sample_results['lower'],
        sample_results['upper'],
        color='orange',
        alpha=0.4,
        label='90% Prediction Interval',
        zorder=1
    )
    plt.title('CQR Prediction Intervals vs. True Values (Sample of 500, Sorted)')
    plt.xlabel('Sample Index (Sorted by True Value)')
    plt.ylabel('Sale Price')
    plt.legend()
    plt.show()


In [ ]:

    # 10.2. Feature Importance Plots
    fig, axes = plt.subplots(1, 3, figsize=(24, 10))

    # LightGBM Feature Importance
    lgbm_importances = pd.DataFrame({'feature': X.columns, 'importance': lgbm_upper.feature_importances_}).sort_values('importance', ascending=False).head(20)
    sns.barplot(x='importance', y='feature', data=lgbm_importances, ax=axes[0], palette='viridis')
    axes[0].set_title('Top 20 Features (LightGBM)')

    # XGBoost Feature Importance
    xgb_importances = pd.DataFrame({'feature': X.columns, 'importance': xgb_upper.feature_importances_}).sort_values('importance', ascending=False).head(20)
    sns.barplot(x='importance', y='feature', data=xgb_importances, ax=axes[1], palette='plasma')
    axes[1].set_title('Top 20 Features (XGBoost)')

    # CatBoost Feature Importance
    cat_importances = pd.DataFrame({'feature': X.columns, 'importance': cat_upper.feature_importances_}).sort_values('importance', ascending=False).head(20)
    sns.barplot(x='importance', y='feature', data=cat_importances, ax=axes[2], palette='magma')
    axes[2].set_title('Top 20 Features (CatBoost)')

    plt.suptitle('Comparison of Feature Importances Across Models', fontsize=20)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
    # 10.3. Model Agreement Scatter Plots
    fig, axes = plt.subplots(2, 3, figsize=(22, 12))

    # Lower Bound Comparisons
    axes[0, 0].scatter(lgbm_cal_preds_lower, cat_cal_preds_lower, alpha=0.2)
    axes[0, 0].set_title('Lower Bound: LGBM vs CatBoost')
    axes[0, 1].scatter(lgbm_cal_preds_lower, xgb_cal_preds_lower, alpha=0.2)
    axes[0, 1].set_title('Lower Bound: LGBM vs XGBoost')
    axes[0, 2].scatter(cat_cal_preds_lower, xgb_cal_preds_lower, alpha=0.2)
    axes[0, 2].set_title('Lower Bound: CatBoost vs XGBoost')

    # Upper Bound Comparisons
    axes[1, 0].scatter(lgbm_cal_preds_upper, cat_cal_preds_upper, alpha=0.2)
    axes[1, 0].set_title('Upper Bound: LGBM vs CatBoost')
    axes[1, 1].scatter(lgbm_cal_preds_upper, xgb_cal_preds_upper, alpha=0.2)
    axes[1, 1].set_title('Upper Bound: LGBM vs XGBoost')
    axes[1, 2].scatter(cat_cal_preds_upper, xgb_cal_preds_upper, alpha=0.2)
    axes[1, 2].set_title('Upper Bound: CatBoost vs XGBoost')

    for ax_row in axes:
        for ax in ax_row:
            lims = [
                np.min([ax.get_xlim(), ax.get_ylim()]),
                np.max([ax.get_xlim(), ax.get_ylim()]),
            ]
            ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)
            ax.set_aspect('equal', adjustable='box')
            ax.grid(True)

    plt.suptitle('Pairwise Comparison of Model Predictions on Calibration Set (Log Scale)', fontsize=20)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
# 11: Final Training and Submission File Generation
if 'X' in locals():
    print("Retraining all models on the full dataset...")

    lgbm_params_final = {'objective': 'quantile', 'metric': 'quantile', 'random_state': 42, 'n_estimators': lgbm_lower.best_iteration_}
    final_lgbm_lower = lgb.LGBMRegressor(alpha=q_lower, **lgbm_params_final).fit(X, y, categorical_feature=categorical_features)
    lgbm_params_final['n_estimators'] = lgbm_upper.best_iteration_
    final_lgbm_upper = lgb.LGBMRegressor(alpha=q_upper, **lgbm_params_final).fit(X, y, categorical_feature=categorical_features)

    # Prepare full data for final XGBoost training
    X_xgb_full = X.copy()
    X_test_xgb = X_test.copy()
    for col in categorical_features:
        X_xgb_full[col] = X_xgb_full[col].cat.codes
        X_test_xgb[col] = X_test_xgb[col].cat.codes

    # Train final XGBoost models on the full encoded data
    xgb_params_final = {'objective': 'reg:quantileerror', 'random_state': 42, 'n_estimators': 1000, 'tree_method': 'hist'}
    final_xgb_lower = xgb.XGBRegressor(quantile_alpha=q_lower, **xgb_params_final).fit(X_xgb_full, y)
    final_xgb_upper = xgb.XGBRegressor(quantile_alpha=q_upper, **xgb_params_final).fit(X_xgb_full, y)

    # CatBoost final training with controlled logging
    cat_params_final_lower = {'loss_function': f'Quantile:alpha={q_lower}', 'iterations': cat_lower.get_best_iteration(), 'verbose': 0, 'random_seed': 42, 'train_dir': None}
    final_cat_lower = cb.CatBoostRegressor(**cat_params_final_lower).fit(X, y, cat_features=categorical_features)

    cat_params_final_upper = {'loss_function': f'Quantile:alpha={q_upper}', 'iterations': cat_upper.get_best_iteration(), 'verbose': 0, 'random_seed': 42, 'train_dir': None}
    final_cat_upper = cb.CatBoostRegressor(**cat_params_final_upper).fit(X, y, cat_features=categorical_features)

    print("Final model training complete.")

In [ ]:
    # --- Generate Submission ---
    print("Generating predictions for the test set...")
    lgbm_test_lower = final_lgbm_lower.predict(X_test)
    lgbm_test_upper = final_lgbm_upper.predict(X_test)
    xgb_test_lower = final_xgb_lower.predict(X_test_xgb)
    xgb_test_upper = final_xgb_upper.predict(X_test_xgb)
    cat_test_lower = final_cat_lower.predict(X_test)
    cat_test_upper = final_cat_upper.predict(X_test)

    # Ensemble the test predictions
    ensemble_test_lower = np.mean([lgbm_test_lower, xgb_test_lower, cat_test_lower], axis=0)
    ensemble_test_upper = np.mean([lgbm_test_upper, xgb_test_upper, cat_test_upper], axis=0)

    # Apply the correction factor 'q' found during calibration
    final_test_lower_log = ensemble_test_lower - q
    final_test_upper_log = ensemble_test_upper + q

    # Transform back to original scale
    final_test_lower_orig = np.expm1(final_test_lower_log)
    final_test_upper_orig = np.expm1(final_test_upper_log)

    # Post-processing
    final_test_lower_orig = np.maximum(0, final_test_lower_orig)
    final_test_upper_orig = np.maximum(final_test_lower_orig, final_test_upper_orig)

    # Create submission file
    test_df_ids = pd.read_csv(test_path, usecols=['id'])
    submission_df = pd.DataFrame({'id': test_df_ids['id'], 'pi_lower': final_test_lower_orig, 'pi_upper': final_test_upper_orig})

    submission_filename = '20282106_ensemble_submission-v6.csv'
    submission_df.to_csv(submission_filename, index=False)

    print(f"\nSubmission file '{submission_filename}' created successfully!")
    print(f"Saved to: {os.path.abspath(submission_filename)}")

    try:
        display(submission_df.head())
    except NameError:
        print(submission_df.head())

In [ ]:
# 12: Save Models and Preprocessor for Application ---
import joblib
import json
import os

print("Saving models and artifacts...")

# Define the directory to save the artifacts
artifact_path = os.path.join(os.getcwd(), 'ml_model')
os.makedirs(artifact_path, exist_ok=True)

# Save the 6 trained models
joblib.dump(final_lgbm_lower, os.path.join(artifact_path, 'lgbm_lower.pkl'))
joblib.dump(final_lgbm_upper, os.path.join(artifact_path, 'lgbm_upper.pkl'))
joblib.dump(final_xgb_lower, os.path.join(artifact_path, 'xgb_lower.pkl'))
joblib.dump(final_xgb_upper, os.path.join(artifact_path, 'xgb_upper.pkl'))
joblib.dump(final_cat_lower, os.path.join(artifact_path, 'cat_lower.pkl'))
joblib.dump(final_cat_upper, os.path.join(artifact_path, 'cat_upper.pkl'))

# Save the preprocessor pipeline
joblib.dump(preprocessor, os.path.join(artifact_path, 'preprocessor.pkl'))

# Save the feature lists and the 'q' value in a JSON file for easy loading
artifacts_to_save = {
    'q_value': q,
    'numerical_features': numerical_features,
    'categorical_features': categorical_features,
    'features_order': list(X.columns) # Save the exact order of columns
}

with open(os.path.join(artifact_path, 'model_config.json'), 'w') as f:
    json.dump(artifacts_to_save, f)

print(f"All artifacts saved to {artifact_path}")